# The Battle of The Neighbourhoods

## Introduction/Business Understanding

### This project is to give us a glimpse into some of the major neighbourhoods of one of the biggest cities in Canada, namely Toronto. The goal of this project is to help someone who wants to open a shop/business in Toronto determine a suitable location to open there business. By leveraging data science and the Foursquare API they can be provided with details about existing and competing businesses and position their new shop/business accordingly.

### Some of the business requirements of this project include:
### - Determining the top 10 most popular venues in a given neighbourhood
### - Categorizing and providing data on particular venues such as Restaurants, Parks etc. 
### - Producing the locations of the produced venues as required
### - Producing information specific to the category of shop/business that the stakeholder is trying to create
### - and more

### So for this project the stakeholders are business owners looking for locations for their new business/shop. The information generated and produced by this project could also be of interest to various industries. Given the nature of this project and business requirements, we will be choosing a descriptive analytic approach




# Data

## Gathering Data

### We will be gathering data using data provided by Wikipedia on the city of Toronto stored in csv format as well as leveraging the data provided by Foursquare API. Some of the parameters of data we will be using includes 

### - latitude and longitude coordinates of neighbourhoods and venues 
### - top venues in each neighbourhood
### - category of each venue (Restaurant, Park, Museum etc.) 
### - Venue names
### - Neighbourhood names 
### - Venue details etc.

## Example Scenario

### For example lets say an aspiring Restaurant owner would like to open a restaurant somewhere in toronto. This project should help him by providing data regarding similar restaurants in each neighbourhood, their locations (latitude and longitude), and additional details about the restaurant that can be produced by the Foursquare API such as rating, menus, reviews etc.

## Understanding the data

### We can understand various parameters of the data using visual analysis and descriptive statistics. We must also do this to check the data quality (missing, invalid or misleading values) to ensure a comprehensive dataset

## Data Preparation

### This includes aggregating various records of data to create comprehensive single records in a data frame to represent one neighbourhood, with all required parameters as columns


In [2]:
!pip install folium

     |████████████████████████████████| 92kB 6.5MB/s eta 0:00:011


In [3]:

from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Thu, 02 Jan 2020 03:37:39 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Server': 'mw1272.eqiad.wmnet', 'X-Powered-By': 'PHP/7.2.24-1+0~20191026.31+debian9~1.gbpbbacde+wmf1', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Content-Encoding': 'gzip', 'Last-Modified': 'Thu, 02 Jan 2020 03:37:06 GMT', 'Backend-Timing': 'D=104003 t=1577936258920914', 'Content-Length': '15086', 'X-ATS-Timestamp': '1577936481', 'X-Varnish': '183891313 60695028', 'Age': '23720', 'X-Cache': 'cp1081 hit, cp1089 hit/50', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=02-Jan-2020;Path=/;HttpOnly;secure;Expires=Mon, 03 Feb 2020 00:00:00 GMT, WMF-

In [5]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

# lst

In [6]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
print(df.shape)
# df[df.duplicated(['PostalCode'], keep=False)] - this would have shown the duplicate PostalCodes

(210, 3)


In [7]:

df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

In [8]:
df.loc[df['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [9]:
df.shape

(103, 3)

In [11]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2020-01-02 10:13:16--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 169.48.113.194
Connecting to cocl.us (cocl.us)|169.48.113.194|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2020-01-02 10:13:16--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|169.48.113.194|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-02 10:13:19--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-01-02 10:13:19--  https://ibm.box.com/public/static/9afzr83pps4pwf2

## Methodology

### Now that we have loaded our data set into a data frame we perform the following exploratory analysis and inferential statistical testing

### - Adding postal code as a parameter/variable to the data frame
### - Checking whether the data set is correct by displaying records where postal code is 'M5G' and where postal code is 'M9V'
### - We then map the neighborhoods in toronto on a map. (Map works on jupyter notebook but does not render on github)
### - Create app on Foursquare developers tools
### - Leverage data from Foursquare API by making calls with appropriate urls, Client ID and Client secret
### - Display venue data near a given neighbourhood leveraged from Foursquare including Venue name, Venue locality, Venue distance from neighbourhood, Venue address, Venue category etc.
### - Filter this data so that we only include desired parameters such as Venue name, Venue category, latitude and longitude
### - Return the number of these venues returned by Foursquare
### - etc.

In [12]:
dfgeo = pd.read_csv("GeoCord.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [13]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [14]:
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [15]:
df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


In [16]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [21]:
CLIENT_ID = '3V55HIFF5OEDB30PXQ4UBSBXW4LVXEPI3QDYZL1Y0EH5WM4Z' # Foursquare ID
CLIENT_SECRET = 'ZRL1FV0GUENA3A1FLMGZIZEQB4M3MQUZCJLZBG5JDELQ0CIB' # Foursquare Secret
VERSION = '20180605' # API version

In [22]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=3V55HIFF5OEDB30PXQ4UBSBXW4LVXEPI3QDYZL1Y0EH5WM4Z&client_secret=ZRL1FV0GUENA3A1FLMGZIZEQB4M3MQUZCJLZBG5JDELQ0CIB&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [24]:
results = requests.get(url).json()
# results

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']

In [27]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c633939e1621b8d48842553-0,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",4c633939e1621b8d48842553,6210 Finch Ave. W Suite 103,CA,Etobicoke,Canada,at Albion Rd.,344,"[6210 Finch Ave. W Suite 103 (at Albion Rd.), ...","[{'label': 'display', 'lat': 43.74242142838322...",43.742421,-79.589471,M9V 3Y5,ON,Subway,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be58dc4cf200f479154133c-1,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",4be58dc4cf200f479154133c,1530 Albion Rd,CA,Etobicoke,Canada,Albion Mall,405,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'label': 'display', 'lat': 43.741685, 'lng':...",43.741685,-79.584487,M9V 1B4,ON,Shoppers Drug Mart,0,[]
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4be70e26cf200f47e334153c-2,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",4be70e26cf200f47e334153c,1530 Albion Rd.,CA,Etobicoke,Canada,at Kipling Ave. (Albion Centre),370,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'label': 'display', 'lat': 43.74120165509377...",43.741202,-79.584545,M9V 1B4,ON,Popeyes Louisiana Kitchen,0,[]
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b04a05bf964a520c45522e3-3,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",4b04a05bf964a520c45522e3,1530 Albion Rd,CA,Etobicoke,Canada,at Finch Ave. W.,413,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'label': 'display', 'lat': 43.741696, 'lng':...",43.741696,-79.584379,M9V 1B4,ON,Sheriff's No Frills,0,[]
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4cd4738cdfb4a1cd4337535c-4,"[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",4cd4738cdfb4a1cd4337535c,1530 Albion Rd,CA,Etobicoke,Canada,Near Finch Ave. W.,413,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'label': 'display', 'lat': 43.7416936, 'lng'...",43.741694,-79.584373,M9V 1B4,ON,The Beer Store,0,[]
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4d8ba6910c4e41bdaaf7667f-5,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",4d8ba6910c4e41bdaaf7667f,"1530 Albion Road, Unit T25",CA,Etobicoke,Canada,NaN,397,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'label': 'display', 'lat': 43.74156896801906...",43.741569,-79.584489,M9V 1B4,ON,Pizza Pizza,0,[]
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5112b872e4b0c0a78d7bcd27-6,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",5112b872e4b0c0a78d7bcd27,1620 Albion road,CA,Toronto,Canada,Albion Road and Finch Ave,319,"[1620 Albion road (Albion Road and Finch Ave),...","[{'label': 'display', 'lat': 43.74184023292111...",43.741840,-79.590561,NaN,ON,Sunny Foodmart,0,[]
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c1951d6834e2d7f2d3a2a80-7,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",4c1951d6834e2d7f2d3a2a80,"1530 Albion Road, Unit F-1",CA,Toronto,Canada,NaN,427,"[1530 Albion Road, Unit F-1, Toronto ON M9V 1B...","[{'label': 'display', 'lat': 43.7417571, 'lng'...",43.741757,-79.584230,M9V 1B4,ON,McDonald's,0,[]


In [28]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Subway,Sandwich Place,43.742421,-79.589471
1,Shoppers Drug Mart,Pharmacy,43.741685,-79.584487
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
3,Sheriff's No Frills,Grocery Store,43.741696,-79.584379
4,The Beer Store,Beer Store,43.741694,-79.584373


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


In [32]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])


Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [33]:
print(venues.shape)
venues.head()


(2222, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Scarborough,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [34]:
venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,1,1,1,1,1,1
Afghan Restaurant,1,1,1,1,1,1
Airport,2,2,2,2,2,2
Airport Food Court,1,1,1,1,1,1
Airport Gate,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2
Airport Service,1,1,1,1,1,1
Airport Terminal,2,2,2,2,2,2
American Restaurant,25,25,25,25,25,25


In [36]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 273 unique categories.


In [35]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,116,116,116,116,116,116
Downtown Toronto,1312,1312,1312,1312,1312,1312
East Toronto,125,125,125,125,125,125
East York,73,73,73,73,73,73
Etobicoke,69,69,69,69,69,69
Mississauga,11,11,11,11,11,11
North York,244,244,244,244,244,244
Scarborough,90,90,90,90,90,90
West Toronto,164,164,164,164,164,164


In [37]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
onehot.shape

(2222, 273)

In [39]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,...,0.008621,0.000000,0.000000,0.008621,0.000000,0.000000,0.000000,0.000000,0.000000,0.008621
1,Downtown Toronto,0.000000,0.000762,0.000762,0.000762,0.000762,0.001524,0.000762,0.001524,0.012195,...,0.012957,0.001524,0.000000,0.005335,0.000000,0.006860,0.000762,0.001524,0.001524,0.003049
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.008000,0.000000,0.000000,0.000000,0.016000
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.013699,0.000000,0.013699,0.000000,0.000000,0.000000,0.000000,0.013699
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004098,0.000000,0.004098,0.000000,0.000000,0.000000,0.000000,0.000000,0.008197,...,0.000000,0.004098,0.004098,0.008197,0.000000,0.000000,0.000000,0.004098,0.016393,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,...,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.018293,0.000000,0.000000,0.012195,0.000000,0.006098,0.000000,0.000000,0.000000,0.012195
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000


In [40]:
grouped.shape

(10, 273)

In [41]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Park  0.06
3            Café  0.05
4  Clothing Store  0.04


----Downtown Toronto----
          venue  freq
0   Coffee Shop  0.10
1          Café  0.05
2         Hotel  0.03
3    Restaurant  0.03
4  Burger Joint  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.04
4             Brewery  0.03


----East York----
          venue  freq
0   Coffee Shop  0.08
1  Burger Joint  0.05
2          Bank  0.04
3      Pharmacy  0.04
4   Pizza Place  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.12
1        Sandwich Place  0.07
2              Pharmacy  0.06
3                   Gym  0.04
4  Fast Food Restaurant  0.04


----Mississauga----
                       venue  freq
0                Coffee Shop  0.18
1                      Hotel  0.18
2  

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Clothing Store,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Bar,Park,Steakhouse
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Café,Sandwich Place,Restaurant,Pub,American Restaurant
3,East York,Coffee Shop,Burger Joint,Pharmacy,Sporting Goods Shop,Park,Bank,Pizza Place,Indian Restaurant,Supermarket,Beer Store
4,Etobicoke,Pizza Place,Sandwich Place,Pharmacy,Grocery Store,Coffee Shop,Gym,Café,Fast Food Restaurant,Bakery,Park
5,Mississauga,Hotel,Coffee Shop,Gym,Middle Eastern Restaurant,Sandwich Place,American Restaurant,Fried Chicken Joint,Burrito Place,Mediterranean Restaurant,Yoga Studio
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Japanese Restaurant,Park,Restaurant,Grocery Store,Pizza Place,Sushi Restaurant,Café
7,Scarborough,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Bakery,Breakfast Spot,Pizza Place,Pharmacy,Middle Eastern Restaurant,Bank,Park
8,West Toronto,Bar,Café,Coffee Shop,Restaurant,Italian Restaurant,Bakery,Diner,Pizza Place,Music Venue,Asian Restaurant
9,York,Park,Fast Food Restaurant,Convenience Store,Trail,Pizza Place,Restaurant,Sandwich Place,Bus Line,Market,Field


In [44]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 3, 4, 0, 3, 3, 2, 1], dtype=int32)

In [45]:
merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017241,...,Coffee Shop,Sandwich Place,Park,Café,Clothing Store,Pizza Place,Dessert Shop,Sushi Restaurant,Restaurant,Pub
1,Downtown Toronto,0.0,0.000762,0.000762,0.000762,0.000762,0.001524,0.000762,0.001524,0.012195,...,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Bar,Park,Steakhouse
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024000,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Brewery,Café,Sandwich Place,Restaurant,Pub,American Restaurant
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Burger Joint,Pharmacy,Sporting Goods Shop,Park,Bank,Pizza Place,Indian Restaurant,Supermarket,Beer Store
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Pizza Place,Sandwich Place,Pharmacy,Grocery Store,Coffee Shop,Gym,Café,Fast Food Restaurant,Bakery,Park


# Results

### We have successfully provided all the data required by business owners to allow them to open their new businesses in the most strategic and beneficial areas. We have met all our business requirements. We have provided our stakeholders with the data required in a filtered, structured and easy to understand format. 

### We have provided all sorts of relevant data statistics such as most popular venues in each respective neighbourhood, most popular venues in a given neighbourhood, additional details of each venue leveraged by Foursquare including addresses, postal codes, categories, names etc. We have provided venue category specific data as well.